# Counting Values

> Counting unique elements in a sequence in various ways (absolute and cumulative, count and percentage).

In [ ]:
#| default_exp value_counts_plus

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import pandas as pd

In [ ]:
#| export
def value_counts_plus(
    series,
    dropna=False,
    show_top=10,
    sort_others=False,
    style=True,
    background_gradient='cividis'):
    """
    Provide a few ways of showing counts of values of items in ``series``.

    Parameters
    ----------
    series : pandas Series or list
        A sequence of items to count.
    dropna : bool
        Whether or not to drop missing values.
    show_top : int
        How many of the top rows to display.
    sort_others : bool
        Whether or not to place "Others" in the bottom (default) or in its sorted order position
    style : bool
        Whether or not to style values for easier reading. If set to ``True`` the result would
        not be a DataFrame, and cannot be further manipulated. Set the value to ``False`` to
        get aDataFrame as the return value.

    Returns
    -------
    value_counts_df : a pandas.DataFrame showing counts based on the provided arguments
    """
    series = pd.Series(series).rename('data')
    col = 'data'
    val_counts = series.value_counts(dropna=dropna)
    if len(val_counts) > show_top:
        val_counts = pd.concat([
            val_counts[:show_top],
            pd.Series(val_counts[show_top:].sum(), index=['Others:'], name=col)])
        if sort_others:
            val_counts = val_counts.sort_values(ascending=False)
        show_top += 1
    count_df = (val_counts
                .to_frame()
                .assign(cum_count=lambda df: df[col].cumsum(),
                        perc=lambda df: df[col].div(df[col].sum()),
                        cum_perc=lambda df: df['perc'].cumsum())
                .reset_index()
                .rename(columns={'index': col, col: 'count'}))
    if not style:
        return count_df.head(show_top)
    return (count_df.
            head(show_top).style
            .format({'count': '{:,}', 'cumsum': '{:,}', 
                     'perc': '{:.1%}',
                     'cum_count': '{:,}',
                     'cum_perc': '{:.1%}'})
            .background_gradient(background_gradient)
            .highlight_null()
            .set_caption(f'<h2>Counts of <b>{series.name}</b></h2>'))


### Counting a list of status codes - default

In [ ]:
import random
from http import HTTPStatus

import pandas as pd

In [ ]:
status_codes = random.choices(
    [s.value for s in HTTPStatus],
    weights=[.01, 0.04, 0.1] * 20 + [0.1, 0.3],
    k=10000)


In [ ]:
value_counts_plus(status_codes)

,data,count,cum_count,perc,cum_perc
0,511,868,868,8.7%,8.7%
1,424,313,"1,181",3.1%,11.8%
2,303,313,"1,494",3.1%,14.9%
3,451,302,"1,796",3.0%,18.0%
4,201,300,"2,096",3.0%,21.0%
5,421,300,"2,396",3.0%,24.0%
6,508,298,"2,694",3.0%,26.9%
7,307,297,"2,991",3.0%,29.9%
8,401,297,"3,288",3.0%,32.9%
9,416,295,"3,583",2.9%,35.8%


In [ ]:
assert type(value_counts_plus(status_codes)) == pd.io.formats.style.Styler

### Changing the number of displayed rows with `show_top`

In [ ]:
value_counts_plus(status_codes, show_top=15)

,data,count,cum_count,perc,cum_perc
0,511,868,868,8.7%,8.7%
1,424,313,"1,181",3.1%,11.8%
2,303,313,"1,494",3.1%,14.9%
3,451,302,"1,796",3.0%,18.0%
4,201,300,"2,096",3.0%,21.0%
5,421,300,"2,396",3.0%,24.0%
6,508,298,"2,694",3.0%,26.9%
7,307,297,"2,991",3.0%,29.9%
8,401,297,"3,288",3.0%,32.9%
9,416,295,"3,583",2.9%,35.8%


In [ ]:
assert value_counts_plus(status_codes, show_top=15, style=False).shape[0] == 16

### Sorting "Others:" 

In [ ]:
value_counts_plus(status_codes, sort_others=True)

,data,count,cum_count,perc,cum_perc
0,Others:,"6,417","6,417",64.2%,64.2%
1,511,868,"7,285",8.7%,72.9%
2,424,313,"7,598",3.1%,76.0%
3,303,313,"7,911",3.1%,79.1%
4,451,302,"8,213",3.0%,82.1%
5,201,300,"8,513",3.0%,85.1%
6,421,300,"8,813",3.0%,88.1%
7,508,298,"9,111",3.0%,91.1%
8,307,297,"9,408",3.0%,94.1%
9,401,297,"9,705",3.0%,97.0%


In [ ]:
assert value_counts_plus(status_codes, sort_others=True, style=False)['data'].iloc[0] == 'Others:'

### Removing table styling if you want a pure `DataFrame`:

In [ ]:
value_counts_plus(status_codes, style=False)

,data,count,cum_count,perc,cum_perc
0,511,868,868,0.0868,0.0868
1,424,313,1181,0.0313,0.1181
2,303,313,1494,0.0313,0.1494
3,451,302,1796,0.0302,0.1796
4,201,300,2096,0.0300,0.2096
5,421,300,2396,0.0300,0.2396
6,508,298,2694,0.0298,0.2694
7,307,297,2991,0.0297,0.2991
8,401,297,3288,0.0297,0.3288
9,416,295,3583,0.0295,0.3583


In [ ]:
assert type(value_counts_plus(status_codes, style=False)) == pd.DataFrame

### Changing the colormap used with `background_gradient`

In [ ]:
value_counts_plus(status_codes, background_gradient='Greens')

,data,count,cum_count,perc,cum_perc
0,511,868,868,8.7%,8.7%
1,424,313,"1,181",3.1%,11.8%
2,303,313,"1,494",3.1%,14.9%
3,451,302,"1,796",3.0%,18.0%
4,201,300,"2,096",3.0%,21.0%
5,421,300,"2,396",3.0%,24.0%
6,508,298,"2,694",3.0%,26.9%
7,307,297,"2,991",3.0%,29.9%
8,401,297,"3,288",3.0%,32.9%
9,416,295,"3,583",2.9%,35.8%


### Convert the table to raw HTML for embedding in a blog or web page

In [ ]:
print(value_counts_plus(status_codes).to_html()[:700])

<style type="text/css">
#T_5d1ae_row0_col1, #T_5d1ae_row0_col3 {
  background-color: #013271;
  color: #f1f1f1;
}
#T_5d1ae_row0_col2, #T_5d1ae_row0_col4, #T_5d1ae_row1_col1, #T_5d1ae_row1_col3, #T_5d1ae_row2_col1, #T_5d1ae_row2_col3, #T_5d1ae_row3_col1, #T_5d1ae_row3_col3, #T_5d1ae_row4_col1, #T_5d1ae_row4_col3, #T_5d1ae_row5_col1, #T_5d1ae_row5_col3, #T_5d1ae_row6_col1, #T_5d1ae_row6_col3, #T_5d1ae_row7_col1, #T_5d1ae_row7_col3, #T_5d1ae_row8_col1, #T_5d1ae_row8_col3, #T_5d1ae_row9_col1, #T_5d1ae_row9_col3 {
  background-color: #00224e;
  color: #f1f1f1;
}
#T_5d1ae_row1_col2, #T_5d1ae_row1_col4 {
  background-color: #00285b;
  color: #f1f1f1;
}
#T_5d1ae_row2_col2, #T_5d1ae_row2_col4 {
  bac


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()